# Genotyping Pipeline Tutorial (IDAT → GTC → VCF → PLINK) — Bash-Only Notebook

Notebook ini dibuat untuk memahami:
- **input** yang dipakai,
- **command** yang dijalankan,
- **output** yang dihasilkan,
- dan **kenapa** tiap tahap diperlukan.

> Semua langkah dijalankan memakai **bash command** (via `%%bash`)

---

## Prasyarat
Sebelum mulai, pastikan sudah tersedia:
- `iaap-cli`
- `bcftools`
- `plink`
- file chip: `.bpm`, `.csv`, `.egt`
- folder IDAT
- reference genome FASTA
- mapping `asa2rsid.map` dan `chr_map.txt`


## Aturan main
1. Jalankan cell **berurutan**.
2. Setelah tiap step, baca output validasi (`ls`, `head`, `wc`) dan jawab pertanyaannya.
3. Kalau error, stop dan pakai bagian **Troubleshooting** di bawah.


## Step 0 — Setup parameter (disimpan ke `params.sh`)
Karena setiap cell `%%bash` berjalan di shell baru, kita simpan semua parameter di file `params.sh` lalu kita `source` di setiap step.


In [13]:
%%bash
set -euo pipefail

cat > params.sh <<'EOF'

# menyesuaikan path masing-masing
export PATH="/home/sofyan/miniconda3/envs/prs-genesis-env/bin/iaap-cli:$PATH"

# ===== INPUTS =====
MANIFEST_BPM="data/data-reference/ASA-24v1-0_E2.bpm"
MANIFEST_CSV="data/data-reference/ASA-24v1-0_E2.csv"
CLUSTER_EGT="data/data-reference/ASA-24v1-0_A1_ClusterFile.egt"
REF_FASTA="data/data-reference/human_v38.fasta"
IDAT_DIR="data/data-sample"

ASA2RSID="data/data-reference/asa2rsid.map"
CHRMAP="data/data-reference/chr.map"
REF_VERSION="38"

POP_DIR="pop"
POP="INA5"

# ===== OUTPUTS =====
OUT_DIR="output"
THREADS=2
OUT_VCF_PREFIX="out_vcf"
OUT_PLINK_PREFIX="out_plink"
SAMPLE_ID="SAMPLE001"

# ===== DERIVED OUTPUT PATHS =====
OUT_GTC_DIR="${OUT_DIR}/gtc"
OUT_VCF_DIR="${OUT_DIR}/vcf/${REF_VERSION}"
OUT_PLINK_DIR="${OUT_DIR}/plink/${REF_VERSION}"
OUT_ALEL_DIR="${OUT_DIR}/allel/${REF_VERSION}"
EOF

echo "✅ params.sh dibuat. Silakan cek isinya:"
sed -n '1,120p' params.sh


✅ params.sh dibuat. Silakan cek isinya:

# menyesuaikan path masing-masing
export PATH="/home/sofyan/miniconda3/envs/prs-genesis-env/bin/iaap-cli:$PATH"

# ===== INPUTS =====
MANIFEST_BPM="data/data-reference/ASA-24v1-0_E2.bpm"
MANIFEST_CSV="data/data-reference/ASA-24v1-0_E2.csv"
CLUSTER_EGT="data/data-reference/ASA-24v1-0_A1_ClusterFile.egt"
REF_FASTA="data/data-reference/human_v38.fasta"
IDAT_DIR="data/data-sample"

ASA2RSID="data/data-reference/asa2rsid.map"
CHRMAP="data/data-reference/chr.map"
REF_VERSION="38"

POP_DIR="pop"
POP="INA5"

# ===== OUTPUTS =====
OUT_DIR="output"
THREADS=2
OUT_VCF_PREFIX="out_vcf"
OUT_PLINK_PREFIX="out_plink"
SAMPLE_ID="SAMPLE001"

# ===== DERIVED OUTPUT PATHS =====
OUT_GTC_DIR="${OUT_DIR}/gtc"
OUT_VCF_DIR="${OUT_DIR}/vcf/${REF_VERSION}"
OUT_PLINK_DIR="${OUT_DIR}/plink/${REF_VERSION}"
OUT_ALEL_DIR="${OUT_DIR}/allel/${REF_VERSION}"


### Checkpoint (Step 0)
- [ ] File `params.sh` sudah ada dan isinya sesuai folder/file yang kamu punya.
- [ ] `THREADS` sudah diset sesuai komputer (misal 2–4 untuk laptop).

**Pertanyaan**
1) Parameter mana yang termasuk **input data**?  
2) Parameter mana yang termasuk **output setting**?


## Step 0.5 — Buat folder output


In [7]:
%%bash
set -euo pipefail
source params.sh
mkdir -p "$OUT_GTC_DIR" "$OUT_VCF_DIR" "$OUT_PLINK_DIR" "$OUT_ALEL_DIR"

echo "✅ Folder output dibuat:"
echo "  OUT_GTC_DIR  : $OUT_GTC_DIR"
echo "  OUT_VCF_DIR  : $OUT_VCF_DIR"
echo "  OUT_PLINK_DIR: $OUT_PLINK_DIR"
echo "  OUT_ALEL_DIR : $OUT_ALEL_DIR"

ls -lh "$OUT_DIR" || true


✅ Folder output dibuat:
  OUT_GTC_DIR  : output/gtc
  OUT_VCF_DIR  : output/vcf/38
  OUT_PLINK_DIR: output/plink/38
  OUT_ALEL_DIR : output/allel/38
total 0
drwxrwxrwx 1 sofyan sofyan 4.0K Dec 17 23:18 allel
drwxrwxrwx 1 sofyan sofyan 4.0K Dec 17 23:18 gtc
drwxrwxrwx 1 sofyan sofyan 4.0K Dec 17 23:18 plink
drwxrwxrwx 1 sofyan sofyan 4.0K Dec 17 23:18 vcf


### Checkpoint (Step 0.5)
- [ ] Folder `gtc/`, `vcf/`, `plink/`, `allel/` sudah muncul di dalam `OUT_DIR`.

**Pertanyaan**
1) Kenapa kita pisahkan output per tahap (gtc/vcf/plink)?


## Step 0.9 — Cek tools (iaap-cli, bcftools, plink)


In [8]:
%%bash
set -euo pipefail
source params.sh

echo "== TOOL CHECK =="
command -v iaap-cli.sh && iaap-cli.sh --help >/dev/null && echo "✅ iaap-cli OK" || echo "❌ iaap-cli belum ketemu di PATH"
command -v bcftools && bcftools --version | head -n 2 || echo "❌ bcftools belum ada"
command -v plink && plink --version | head -n 2 || echo "❌ plink belum ada"

echo
echo "== FILE CHECK (contoh) =="
ls -lh "$MANIFEST_BPM" "$MANIFEST_CSV" "$CLUSTER_EGT" 2>/dev/null || echo "⚠️ beberapa file chip belum ketemu (cek path di params.sh)"
ls -lh "$IDAT_DIR" 2>/dev/null | head || echo "⚠️ folder IDAT belum ketemu (cek path di params.sh)"


== TOOL CHECK ==
/home/sofyan/miniconda3/envs/prs-genesis-env/bin/iaap-cli/iaap-cli.sh
✅ iaap-cli OK
/home/sofyan/miniconda3/envs/prs-genesis-env/bin/bcftools
bcftools 1.22
Using htslib 1.22.1
/home/sofyan/miniconda3/envs/prs-genesis-env/bin/plink
PLINK v1.9.0-b.8 64-bit (22 Oct 2024)

== FILE CHECK (contoh) ==
-rwxrwxrwx 1 sofyan sofyan 109M Dec 17 21:41 data/data-reference/ASA-24v1-0_A1_ClusterFile.egt
-rwxrwxrwx 1 sofyan sofyan 108M Dec 17 21:49 data/data-reference/ASA-24v1-0_E2.bpm
-rwxrwxrwx 1 sofyan sofyan 273M Dec 17 21:55 data/data-reference/ASA-24v1-0_E2.csv
total 19M
-rwxrwxrwx 1 sofyan sofyan 9.1M Dec 17 10:32 207407180047_R05C02_Grn.idat
-rwxrwxrwx 1 sofyan sofyan 9.1M Dec 17 10:32 207407180047_R05C02_Red.idat
-rwxrwxrwx 1 sofyan sofyan    0 Dec 17 15:07 link_download.txt


### Checkpoint (Step 0.9)
- [ ] `bcftools` dan `plink` terdeteksi.
- [ ] `iaap-cli` terdeteksi (kalau tidak, pastikan PATH sudah benar atau panggil pakai `./iaap-cli`).

**Pertanyaan**
1) Apa bedanya tool “bioinformatika” (bcftools/plink) dengan tool “vendor” (iaap-cli)?


## Step 1 — IDAT → GTC (Genotype Calling)

**Ide besar:**  
IDAT adalah data mentah intensitas dari scanner microarray. `iaap-cli gencall` mengubah intensitas menjadi “genotype calls” (mis. AA/AB/BB) dan menghasilkan file GTC.

### Output utama
- Folder `gtc/` berisi file `.gtc`


In [10]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP1] IDAT -> GTC"
echo "INPUT :"
echo "  IDAT_DIR     = $IDAT_DIR"
echo "  MANIFEST_BPM = $MANIFEST_BPM"
echo "  CLUSTER_EGT  = $CLUSTER_EGT"
echo "OUTPUT:"
echo "  OUT_GTC_DIR  = $OUT_GTC_DIR"
echo "THREADS: $THREADS"
echo

iaap-cli.sh gencall "$MANIFEST_BPM" "$CLUSTER_EGT" "$OUT_GTC_DIR" \
  -f "$IDAT_DIR" -g -t "$THREADS"

echo
echo "✅ Validasi output (contoh 10 file pertama):"
ls -lh "$OUT_GTC_DIR" | head -n 10


[STEP1] IDAT -> GTC
INPUT :
  IDAT_DIR     = data/data-sample
  MANIFEST_BPM = data/data-reference/ASA-24v1-0_E2.bpm
  CLUSTER_EGT  = data/data-reference/ASA-24v1-0_A1_ClusterFile.egt
OUTPUT:
  OUT_GTC_DIR  = output/gtc
THREADS: 2

info: ArrayAnalysis.NormToGenCall.CLI.App[0]
      [03:57:25 3572]: Crawling data/data-sample for samples ...
info: ArrayAnalysis.NormToGenCall.CLI.App[0]
      [03:57:25 3799]: Number of samples to process: 1
info: ArrayAnalysis.NormToGenCall.Services.NormToGenCallSvc[0]
      [03:57:25 3910]: 
      Starting processing...
      Manifest file: data/data-reference/ASA-24v1-0_E2.bpm
      Cluster file: data/data-reference/ASA-24v1-0_A1_ClusterFile.egt
      Include file: 
      Output directory: output/gtc
      GenCall score cutoff: 0.15
      GenTrain ID: 3
      Gender Estimate Settings: 
                      Version: 2
                      MinAutosomalLoci : 100
                      MaxAutosomalLoci : 10000
                      MinXLoci : 20
         

### Checkpoint (Step 1)
- [ ] Ada file di `OUT_GTC_DIR` (biasanya `.gtc`).
- [ ] Kalau kosong: cek `IDAT_DIR` dan pastikan file IDAT lengkap.

**Pertanyaan**
1) Menurut kamu, “genotype calling” itu mengubah data seperti apa menjadi apa?  
2) Kenapa perlu `EGT (cluster file)`?


## Step 2 — GTC → VCF (standar varian)

**Ide besar:**  
VCF adalah format standar untuk menyimpan varian (chr, posisi, ref/alt) + genotype per sampel.

Kita juga melakukan:
- sort (urutkan varian)
- norm (normalisasi terhadap FASTA, buang duplikasi)
- index (buat akses cepat)

### Output utama
- `out_vcf.vcf.gz`
- `out_vcf.vcf.csi` (index)
- `out_vcf.tsv` (extra info dari konversi)


In [11]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP2] GTC -> VCF"
echo "INPUT :"
echo "  OUT_GTC_DIR  = $OUT_GTC_DIR"
echo "  MANIFEST_BPM = $MANIFEST_BPM"
echo "  MANIFEST_CSV = $MANIFEST_CSV"
echo "  CLUSTER_EGT  = $CLUSTER_EGT"
echo "  REF_FASTA    = $REF_FASTA"
echo "OUTPUT:"
echo "  VCF_GZ       = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz"
echo "  VCF_INDEX    = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.csi"
echo "  EXTRA_TSV    = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.tsv"
echo

bcftools +gtc2vcf -Oz --no-version \
  --bpm "$MANIFEST_BPM" \
  --csv "$MANIFEST_CSV" \
  --egt "$CLUSTER_EGT" \
  --gtcs "$OUT_GTC_DIR" \
  --fasta-ref "$REF_FASTA" \
  --extra "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.tsv" | \
  bcftools sort -Oz -T "${OUT_VCF_DIR}/bcftools-sort.XXXXXX" | \
  bcftools norm --no-version --rm-dup all -Oz -c x -f "$REF_FASTA" | \
  tee "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" | \
  bcftools index --force --output "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.csi"

echo
echo "✅ Validasi output:"
ls -lh "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.csi" "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.tsv" 2>/dev/null || true

echo
echo "✅ Preview header VCF (25 baris):"
bcftools view -h "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" | head -n 25


[STEP2] GTC -> VCF
INPUT :
  OUT_GTC_DIR  = output/gtc
  MANIFEST_BPM = data/data-reference/ASA-24v1-0_E2.bpm
  MANIFEST_CSV = data/data-reference/ASA-24v1-0_E2.csv
  CLUSTER_EGT  = data/data-reference/ASA-24v1-0_A1_ClusterFile.egt
  REF_FASTA    = data/data-reference/human_v38.fasta
OUTPUT:
  VCF_GZ       = output/vcf/38/out_vcf.vcf.gz
  VCF_INDEX    = output/vcf/38/out_vcf.vcf.csi
  EXTRA_TSV    = output/vcf/38/out_vcf.tsv



Writing to output/vcf/38/bcftools-sort.XXXXXXYyfXvw
gtc2vcf 2025-01-03 http://github.com/freeseek/gtc2vcf
Reading BPM file data/data-reference/ASA-24v1-0_E2.bpm
Reading CSV file data/data-reference/ASA-24v1-0_E2.csv
Reading EGT file data/data-reference/ASA-24v1-0_A1_ClusterFile.egt
Reading GTC file output/gtc/207407180047_R05C02.gtc
Writing VCF file
Lines   total/missing-reference/skipped:	657490/1/599
Merging 1 temporary files
Done
Cleaning
Lines   total/split/joined/realigned/mismatch_removed/dup_removed/skipped:	656891/0/0/754/1/9126/0



✅ Validasi output:
-rwxrwxrwx 1 sofyan sofyan  779 Dec 18 04:21 output/vcf/38/out_vcf.tsv
-rwxrwxrwx 1 sofyan sofyan 1.3M Dec 18 04:22 output/vcf/38/out_vcf.vcf.csi
-rwxrwxrwx 1 sofyan sofyan  82M Dec 18 04:22 output/vcf/38/out_vcf.vcf.gz

✅ Preview header VCF (25 baris):
##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##contig=<ID=chr1,length=248956422>
##contig=<ID=chr2,length=242193529>
##contig=<ID=chr3,length=198295559>
##contig=<ID=chr4,length=190214555>
##contig=<ID=chr5,length=181538259>
##contig=<ID=chr6,length=170805979>
##contig=<ID=chr7,length=159345973>
##contig=<ID=chr8,length=145138636>
##contig=<ID=chr9,length=138394717>
##contig=<ID=chr10,length=133797422>
##contig=<ID=chr11,length=135086622>
##contig=<ID=chr12,length=133275309>
##contig=<ID=chr13,length=114364328>
##contig=<ID=chr14,length=107043718>
##contig=<ID=chr15,length=101991189>
##contig=<ID=chr16,length=90338345>
##contig=<ID=chr17,length=83257441>
##contig=<ID=chr18,length=80373285>

### Checkpoint (Step 2)
- [ ] File `.vcf.gz` dan `.vcf.csi` ada.
- [ ] Kamu bisa lihat header VCF muncul.

**Pertanyaan**
1) Kenapa VCF perlu di-**index**?  
2) Menurutmu fungsi `sort` dan `norm` apa?  
3) Di header VCF, cari baris yang berisi `##reference` atau info referensi. Ada nggak?


## Step 3 — VCF → PLINK (bed/bim/fam)

**Ide besar:**  
PLINK format cepat untuk analisis genetik. Kita ubah VCF jadi:
- `.bed` (binary genotype matrix)
- `.bim` (daftar varian)
- `.fam` (daftar sampel)

Di tahap ini juga:
- memecah multi-allelic → biallelic
- menormalkan lagi terhadap FASTA
- rename kromosom via `CHRMAP`
- memberi ID varian format `CHROM:POS:REF:ALT` supaya unik


In [14]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP3] VCF -> PLINK"
echo "INPUT :"
echo "  VCF_GZ   = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz"
echo "  REF_FASTA= $REF_FASTA"
echo "  CHRMAP   = $CHRMAP"
echo "OUTPUT:"
echo "  PLINK    = ${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.{bed,bim,fam}"
echo

bcftools norm -Ou -m -any "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" | \
  bcftools norm -Ou -f "$REF_FASTA" | \
  bcftools annotate --rename-chrs "$CHRMAP" | \
  bcftools annotate -Ob -I '%CHROM:%POS:%REF:%ALT' | \
  plink --bcf /dev/stdin \
    --keep-allele-order \
    --vcf-idspace-to _ \
    --double-id \
    --allow-extra-chr 0 \
    --split-x b38 no-fail \
    --make-bed \
    --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}"

echo
echo "✅ Validasi output:"
ls -lh "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bed" "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bim" "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam" 2>/dev/null || true

echo
echo "✅ Preview .fam (5 baris):"
head -n 5 "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam"

echo
echo "✅ Preview .bim (5 baris):"
head -n 5 "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bim"


[STEP3] VCF -> PLINK
INPUT :
  VCF_GZ   = output/vcf/38/out_vcf.vcf.gz
  REF_FASTA= data/data-reference/human_v38.fasta
  CHRMAP   = data/data-reference/chr.map
OUTPUT:
  PLINK    = output/plink/38/out_plink.{bed,bim,fam}



PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/plink/38/out_plink.log.
Options in effect:
  --allow-extra-chr 0
  --bcf /dev/stdin
  --double-id
  --keep-allele-order
  --make-bed
  --out output/plink/38/out_plink
  --split-x b38 no-fail
  --vcf-idspace-to _

7798 MB RAM detected; reserving 3899 MB for main workspace.
--bcf: 645k variants complete.

Lines   total/split/joined/realigned/mismatch_removed/dup_removed/skipped:	647764/0/0/0/0/0/0


--bcf: 646k variants complete.

Lines   total/split/joined/realigned/mismatch_removed/dup_removed/skipped:	647764/0/0/0/0/0/0


--bcf: output/plink/38/out_plink-temporary.bed +
output/plink/38/out_plink-temporary.bim +
output/plink/38/out_plink-temporary.fam written.


647764 variants loaded from .bim file.
1 person (0 males, 0 females, 1 ambiguous) loaded from .fam.
Ambiguous sex ID written to output/plink/38/out_plink.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1 founder and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.


treat these as missing.


Total genotyping rate is 0.995884.
647764 variants and 1 person pass filters and QC.
Note: No phenotypes present.
--split-x: 290 chromosome codes changed.
--make-bed to output/plink/38/out_plink.bed + output/plink/38/out_plink.bim +
output/plink/38/out_plink.fam ... 1010111112121313141415151616171718181919202021212222232324252526262727282829293030313132323333343435353636373738383939404041414242434344444545464647474848495050515152525353545455555656575758585959606061616262636364646565666667676868696970707171727273737475757676777778787979808081818282838384848585868687878888898990909191929293939494959596969797989899done.

✅ Validasi output:
-rwxrwxrwx 1 sofyan sofyan 633K Dec 18  2025 output/plink/38/out_plink.bed
-rwxrwxrwx 1 sofyan sofyan  21M Dec 18  2025 output/plink/38/out_plink.bim
-rwxrwxrwx 1 sofyan sofyan   49 Dec 18  2025 output/plink/38/out_plink.fam

✅ Preview .fam (5 baris):
207407180047_R05C02 207407180047_R05C02 0 0 0 -9

✅ Preview .bim (5 baris):
1	1:629241:C:T	0	629241	T	C

### Checkpoint (Step 3)
- [ ] File `.bed`, `.bim`, `.fam` ada.
- [ ] Kamu bisa lihat contoh isi `.fam` dan `.bim`.

**Pertanyaan**
1) Dari `.fam`, kolom mana yang terlihat seperti “ID sampel”?  
2) Dari `.bim`, kolom mana yang terlihat seperti “posisi varian”?  
3) Kenapa kita pilih ID varian `CHROM:POS:REF:ALT` (bukan rsID dulu)?


## Step 4 — Update Sample ID

**Ide besar:**  
Kadang ID sampel dari proses konversi bukan nama yang kita inginkan. Kita ganti supaya rapih dan konsisten.

Output yang dicek:
- `.fam` harus menunjukkan `SAMPLE_ID` yang kamu set.


In [15]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP4] Update Sample ID"
echo "TARGET SAMPLE_ID: $SAMPLE_ID"

IDAT_ID=$(awk '{print $1; exit}' "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam")

echo "IDAT_ID (lama)  : $IDAT_ID"
echo "SAMPLE_ID (baru): $SAMPLE_ID"

echo -e "${IDAT_ID}\t${IDAT_ID}\t${SAMPLE_ID}\t${SAMPLE_ID}" > "${OUT_PLINK_DIR}/sample.map"

plink --bfile "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}" \
  --update-ids "${OUT_PLINK_DIR}/sample.map" \
  --make-bed --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}"

echo
echo "✅ Preview .fam setelah update (5 baris):"
head -n 5 "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam"


[STEP4] Update Sample ID
TARGET SAMPLE_ID: SAMPLE001
IDAT_ID (lama)  : 207407180047_R05C02
SAMPLE_ID (baru): SAMPLE001
PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/plink/38/out_plink.log.
Options in effect:
  --bfile output/plink/38/out_plink
  --make-bed
  --out output/plink/38/out_plink
  --update-ids output/plink/38/sample.map

7798 MB RAM detected; reserving 3899 MB for main workspace.


filenames.


647764 variants loaded from .bim file.
1 person (0 males, 0 females, 1 ambiguous) loaded from .fam.
Ambiguous sex ID written to output/plink/38/out_plink.nosex .
--update-ids: 1 person updated.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1 founder and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.


treat these as missing.


Total genotyping rate is 0.995884.
647764 variants and 1 person pass filters and QC.
Note: No phenotypes present.
--make-bed to output/plink/38/out_plink.bed + output/plink/38/out_plink.bim +
output/plink/38/out_plink.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.

✅ Preview .fam setelah update (5 baris):
SAMPLE001 SAMPLE001 0 0 0 -9


### Checkpoint (Step 4)
- [ ] `.fam` sekarang menampilkan ID yang kamu harapkan.

**Pertanyaan**
1) Menurutmu kenapa “penamaan sampel” penting dalam pipeline bioinformatika?  
2) Apa risiko kalau nama sampel berubah-ubah di setiap tahap?


## Step 5 — Membuat `variant.list` (ringkasan varian + genotype)

**Ide besar:**  
Dari VCF, kita buat list sederhana:
- ID varian (di-map ke rsID jika ada di `ASA2RSID`)
- genotype (mis. 0/1, 1/1, dll)

Output:
- `allel/<ref_version>/variant.list`

> Catatan: di sini kita memakai `process substitution <( ... )`, jadi kita jalankan dalam bash.


In [17]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP6] Create variant.list"
echo "OUTPUT: ${OUT_ALEL_DIR}/variant.list"
echo

awk -F'\t' 'NR==FNR {a[$1]=$2; next} { if($1 in a) {print a[$1]"\t"$2} else {print $0} }' "$ASA2RSID" \
  <(awk -F'\t' '/^[^#]/{split($10,a,":"); gsub(/^chr/, "", $1); print $1":"$2":"$4":"$5"\t"a[1]}' \
    <(gunzip -c "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz")) \
  > "${OUT_ALEL_DIR}/variant.list"

echo "✅ Preview variant.list (10 baris):"
head -n 10 "${OUT_ALEL_DIR}/variant.list"

echo
echo "✅ Hitung jumlah baris:"
wc -l "${OUT_ALEL_DIR}/variant.list"


[STEP6] Create variant.list
OUTPUT: output/allel/38/variant.list

✅ Preview variant.list (10 baris):
rs10458597:C:T	0/0
rs9651229:C:T	0/0
rs9701872:T:C	0/0
rs11497407:G:A	0/0
rs9645429:G:A	0/0
rs189147642:T:A	0/0
rs3131972:A:G	1/1
rs192434564:A:G	0/0
rs78408995:C:T	0/0
rs80302046:G:A	0/0

✅ Hitung jumlah baris:
647764 output/allel/38/variant.list


### Checkpoint (Step 5)
- [ ] `variant.list` berhasil dibuat.
- [ ] Kamu bisa melihat pasangan “ID → genotype”.

**Pertanyaan**
1) Apa keuntungan membuat `variant.list` dibanding membaca VCF langsung?  
2) Kalau genotype `0/0`, `0/1`, `1/1` artinya apa?

**Mini-Challenge**
- Cari 1 baris `variant.list` dan jelaskan artinya dalam kalimat sederhana.


## Step 6 — Update SNP ID ke rsID + export `out23.txt`

**Ide besar:**  
- `--update-name` mengganti ID varian di PLINK menggunakan mapping `ASA2RSID` (jika tersedia).
- `out23.txt` adalah tabel ringkas ala “genotype report” (bukan format 23andMe resmi).

Output:
- `vcf/<ref_version>/out23.txt`


In [18]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP7] Update SNPID + Export out23.txt"
echo "OUTPUT: ${OUT_VCF_DIR}/out23.txt"
echo

plink --bfile "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}" \
  --update-name "$ASA2RSID" --make-bed \
  --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}"

awk -F'\t' '{split($2,a,":"); print a[1]"\t"$1"\t"$4"\t"$6$5}' \
  "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bim" > "${OUT_VCF_DIR}/out23.txt"

echo "✅ Preview out23.txt (10 baris):"
head -n 10 "${OUT_VCF_DIR}/out23.txt"


[STEP7] Update SNPID + Export out23.txt
OUTPUT: output/vcf/38/out23.txt

PLINK v1.9.0-b.8 64-bit (22 Oct 2024)              cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to output/plink/38/out_plink.log.
Options in effect:
  --bfile output/plink/38/out_plink
  --make-bed
  --out output/plink/38/out_plink
  --update-name data/data-reference/asa2rsid.map

7798 MB RAM detected; reserving 3899 MB for main workspace.


filenames.
your file and command-line parameters, and consider changing your naming
scheme if you encounter memory problems.


647764 variants loaded from .bim file.
1 person (0 males, 0 females, 1 ambiguous) loaded from .fam.
Ambiguous sex ID written to output/plink/38/out_plink.nosex .
--update-name: 629876 values updated, 49379 variant IDs not present.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1 founder and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.


treat these as missing.


Total genotyping rate is 0.995884.
647764 variants and 1 person pass filters and QC.
Note: No phenotypes present.
--make-bed to output/plink/38/out_plink.bed + output/plink/38/out_plink.bim +
output/plink/38/out_plink.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
✅ Preview out23.txt (10 baris):
rs10458597	1	629241	CT
rs9651229	1	632287	CT
rs9701872	1	632828	TC
rs11497407	1	633147	GA
rs9645429	1	634224	GA
rs189147642	1	786377	TA
rs3131972	1	817341	GA
rs192434564	1	818818	AG
rs78408995	1	819894	CT
rs80302046	1	821576	GA


### Checkpoint (Step 6)
- [ ] `out23.txt` ada dan bisa dibaca.
- [ ] Kolom-kolomnya terlihat seperti: rsID, chromosome, position, genotype.

**Pertanyaan**
1) Kenapa kadang rsID bisa bermasalah (duplicate/ambigu) sehingga pipeline sebelumnya pakai `CHROM:POS:REF:ALT`?  
2) Apa bedanya “ID varian” vs “posisi varian”?

**Mini-Project (opsional)**
- Ambil 5 baris `out23.txt` dan buat “mini laporan”:
  - varian apa,
  - berada di kromosom berapa,
  - genotipe apa.


# Troubleshooting cepat

Kalau ada error, cek hal berikut (jalankan cell sesuai kebutuhan):
- path file salah (`params.sh`)
- tool belum ada di PATH
- permission (`chmod +x iaap-cli`)
- file IDAT kurang lengkap
- referensi FASTA beda build (GRCh37 vs GRCh38)


In [19]:
%%bash
set -euo pipefail
set -euo pipefail
echo "== QUICK CHECK: current dir & files =="
pwd
ls -lh | head -n 30

echo
echo "== QUICK CHECK: params.sh =="
test -f params.sh && echo "✅ params.sh exists" || echo "❌ params.sh missing"


== QUICK CHECK: current dir & files ==
/mnt/d/projek/genesis
total 52K
-rwxrwxrwx 1 sofyan sofyan  494 Dec 17 14:41 README.md
drwxrwxrwx 1 sofyan sofyan 4.0K Dec 17 14:51 data
-rwxrwxrwx 1 sofyan sofyan  43K Dec 18  2025 genotyping_pipeline_bash_tutorial.ipynb
drwxrwxrwx 1 sofyan sofyan 4.0K Dec 18  2025 output
-rwxrwxrwx 1 sofyan sofyan  836 Dec 18 04:27 params.sh
-rwxrwxrwx 1 sofyan sofyan 1.3K Dec 17 16:51 prs-genesis-env.yml
drwxrwxrwx 1 sofyan sofyan 4.0K Dec 17 20:44 tools

== QUICK CHECK: params.sh ==
✅ params.sh exists


In [20]:
%%bash
set -euo pipefail
set -euo pipefail
source params.sh

echo "== TOOL PATH =="
command -v iaap-cli || true
command -v bcftools || true
command -v plink || true

echo
echo "== INPUT FILES EXIST? =="
for f in "$MANIFEST_BPM" "$MANIFEST_CSV" "$CLUSTER_EGT" "$ASA2RSID" "$CHRMAP"; do
  if [ -f "$f" ]; then echo "✅ $f"; else echo "❌ missing: $f"; fi
done

echo
echo "== IDAT DIR EXIST? =="
if [ -d "$IDAT_DIR" ]; then
  echo "✅ $IDAT_DIR"
  ls -lh "$IDAT_DIR" | head
else
  echo "❌ missing dir: $IDAT_DIR"
fi

echo
echo "== OUTPUT FILES (if already ran some steps) =="
ls -lh "$OUT_GTC_DIR" 2>/dev/null | head || true
ls -lh "$OUT_VCF_DIR" 2>/dev/null | head || true
ls -lh "$OUT_PLINK_DIR" 2>/dev/null | head || true


== TOOL PATH ==
/home/sofyan/miniconda3/envs/prs-genesis-env/bin/iaap-cli/iaap-cli
/home/sofyan/miniconda3/envs/prs-genesis-env/bin/bcftools
/home/sofyan/miniconda3/envs/prs-genesis-env/bin/plink

== INPUT FILES EXIST? ==
✅ data/data-reference/ASA-24v1-0_E2.bpm
✅ data/data-reference/ASA-24v1-0_E2.csv
✅ data/data-reference/ASA-24v1-0_A1_ClusterFile.egt
✅ data/data-reference/asa2rsid.map
✅ data/data-reference/chr.map

== IDAT DIR EXIST? ==
✅ data/data-sample
total 19M
-rwxrwxrwx 1 sofyan sofyan 9.1M Dec 17 10:32 207407180047_R05C02_Grn.idat
-rwxrwxrwx 1 sofyan sofyan 9.1M Dec 17 10:32 207407180047_R05C02_Red.idat
-rwxrwxrwx 1 sofyan sofyan    0 Dec 17 15:07 link_download.txt

== OUTPUT FILES (if already ran some steps) ==
total 12M
-rwxrwxrwx 1 sofyan sofyan 12M Dec 18 04:07 207407180047_R05C02.gtc
total 99M
-rwxrwxrwx 1 sofyan sofyan  16M Dec 18  2025 out23.txt
-rwxrwxrwx 1 sofyan sofyan  779 Dec 18 04:21 out_vcf.tsv
-rwxrwxrwx 1 sofyan sofyan 1.3M Dec 18 04:22 out_vcf.vcf.csi
-rwxrwxrw